## Piston simulation function

Using linear regression, NN for input-output relationships is called black-bod modelling. In many practical applications, we have some knowledge of the input-output model in terms of equations (white-box model). The goal of this notebook is to learn how 
1. to estimate the parameters of these equations from data
2. to combine equations and Machine Learning to compensate modelling errors.
3. compare pure black-box, white-box, and mixed models for predictions.

We will use again the Piston Simulation function, which models the circular motion of a piston within a cylinder. It involves a chain of nonlinear functions, see [here](https://www.sfu.ca/~ssurjano/piston.html) for the equations.

The response C is cycle time (the time it takes to complete one cycle), in seconds. 

The input variables and their usual input ranges are:

* M in [30, 60] 	piston weight (kg)
* S in [0.005, 0.020] 	piston surface area (m2)
* V0 in [0.002, 0.010] 	initial gas volume (m3)
* k in [1000, 5000] 	spring coefficient (N/m)
* P0 in [90000, 110000]    	atmospheric pressure (N/m2)
* Ta in [290, 296]    	ambient temperature (K)
* T0 in [340, 360]    	filling gas temperature (K)


In [28]:
#this function implements the equations
import numpy as np

def piston(xx):

    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #%
    #% OUTPUT AND INPUT:
    #%
    #% C = cycle time
    #% xx = [M, S, V0, k, P0, Ta, T0]
    #%
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

    M  = xx[0];
    S  = xx[1];
    V0 = xx[2];
    k  = xx[3];
    P0 = xx[4];
    Ta = xx[5];
    T0 = xx[6];

    Aterm1 = P0 * S;
    Aterm2 = 19.62 * M;
    Aterm3 = -k*V0 / S;
    A = Aterm1 + Aterm2 + Aterm3;

    Vfact1 = S / (2*k);
    Vfact2 = np.sqrt(A**2 + 4*k*(P0*V0/T0)*Ta);
    V = Vfact1 * (Vfact2 - A);

    fact1 = M;
    fact2 = k + (S**2)*(P0*V0/T0)*(Ta/(V**2));

    C = 2 * np.pi * np.sqrt(fact1/fact2);

    return C


# We make a dataset
n=200
M=np.random.uniform(30, 60, n)[:,None] #piston weight (kg)
S=np.random.uniform(0.005, 0.020, n)[:,None] #piston surface area (m2)
V0=0.005*np.ones(n)[:,None]; #UNKNOWN 
k=3500*np.ones(n)[:,None] #UNKNOWN 
P0=np.random.uniform(90000, 110000, n)[:,None] #atmospheric pressure (N/m2)
Ta=np.random.uniform(290, 296, n)[:,None] #ambient temperature (K)
T0=347*np.ones(n)[:,None]#UNKNOWN 

np.random.seed(42)
X=np.hstack([M,S,V0,k,P0,Ta,T0])#dataset
Y=[]
for i in range(n):
    Y.append(piston(X[i,:])+np.random.randn(1)*0.01)#we add some Gaussian noise
Y=np.array(Y)

X=X[:,[0,1,4,5]]#we drop the columns we do not know. This is your input.

## Estimating the parameters of process equations

When we derive a mathematical model of some process (chemical, manufacturing, etc.), some of the parameters may be unknown.
For instance, in the Piston example, these parameters are
* spring coefficient (N/m)
* initial gas volume (m3)
* filling gas temperature (K)

To estimate these parameters we can perform a DOE (design of experiments). It helps you investigate the effects of input variables (factors we can change) on an output variable (response). These experiments consist of a series of runs, or tests, in which purposeful changes are made to the input variables.
In the code above, I have just generate a random design, but there are other better ways, see [PyDOE](https://pythonhosted.org/pyDOE/).

**Question 1:** Assuming the values of $V0,K,T0$ are unknown, use the above data from the DOE to estimate the value of these three unknown parameters. In particular, write a PyMC3 models that compute the posterior of $V0,K,T0$ and then again predict the value of $C$ at $Xpred=[45,0.015,0.008,3500,100000,290,341]$. We assume that you know the equations above of the piston.

In [24]:

def piston_modified(xx):

    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #%
    #% OUTPUT AND INPUT:
    #%
    #% C = cycle time
    #% xx = [M, S, V0, k, P0, Ta, T0]
    #%
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

    M  = xx[0];
    S  = xx[1];
    V0 = xx[2];
    k  = xx[3];
    P0 = xx[4];
    Ta = xx[5];
    T0 = xx[6];

    Aterm1 = P0 * S;
    Aterm2 = 19.62 * M;
    Aterm3 = -k*V0 / S;
    A = Aterm1 + Aterm2 + Aterm3;

    Vfact1 = S / (2*k);
    Vfact2 = np.sqrt(A**2 + 4*k*(P0*V0/T0)*Ta);
    V = Vfact1 * (Vfact2 - A);

    fact1 = M;
    fact2 = k + (S**2)*(P0*V0/T0)*(Ta/(abs(V)**1.7));

    C = 2 * np.pi * np.sqrt(fact1/fact2);

    return C


**Question 2:** Sometimes the model we derive is not very good. For instance, assume we have written the model `piston_modified` for the Piston, but instead the "true" model is `piston`.

In [25]:
Xtest=np.array([[45,0.015,0.008,3500,100000,290,341]])
print(piston(Xtest[0,:]),piston_modified(Xtest[0,:]))

0.4603480963134951 0.6261879786383996


Consider the dataset above, but this time assume that you know the equations in `piston_modified` but not the equations in `piston` and as before you do not know $V0,K,T0$.   Your goal is to build a model that combines `piston_modified`  with a Bayesian NN to compensate for modelling errors. Answer these questions?
1. using the previous training dataset, predict C for the test set below using `piston_modified` alone
2. using the previous training dataset, predict C for the test set below using a Bayesian NN alone
3. using the previous training dataset, predict C for the test set below using a model that combines `piston_modified`  with a Bayesian NN
4. evaluate the prediction error, compare the three models and also compare their prediction at Xtest=np.array([[45,0.015,0.008,3500,100000,290,341]])



In [30]:

# We make a test dataset
n=400
M=np.random.uniform(30, 60, n)[:,None] #piston weight (kg)
S=np.random.uniform(0.005, 0.020, n)[:,None] #piston surface area (m2)
V0=0.005*np.ones(n)[:,None]; #UNKNOWN 
k=3500*np.ones(n)[:,None] #UNKNOWN 
P0=np.random.uniform(90000, 110000, n)[:,None] #atmospheric pressure (N/m2)
Ta=np.random.uniform(290, 296, n)[:,None] #ambient temperature (K)
T0=347*np.ones(n)[:,None]#UNKNOWN 

np.random.seed(100)
Xte=np.hstack([M,S,V0,k,P0,Ta,T0])#dataset
Yte=[]
for i in range(n):
    Yte.append(piston(Xte[i,:])+np.random.randn(1)*0.01)#we add some Gaussian noise
Yte=np.array(Yte)

Xte=Xte[:,[0,1,4,5]]#we drop the columns we do not know. This is test set input.
Xte

array([[3.83808653e+01, 1.38666961e-02, 9.15863759e+04, 2.92783750e+02],
       [5.88311290e+01, 1.26914693e-02, 9.90470313e+04, 2.92444115e+02],
       [3.29439647e+01, 5.37188661e-03, 9.88431149e+04, 2.95948475e+02],
       ...,
       [3.28710930e+01, 1.97931696e-02, 9.11207319e+04, 2.90825935e+02],
       [5.16603217e+01, 1.32017445e-02, 9.94360233e+04, 2.93520831e+02],
       [4.18096556e+01, 1.90887090e-02, 1.02619836e+05, 2.95698462e+02]])